# STEP MODEL

## GET DATA

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.linear_model import RidgeCV
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
import warnings
from tqdm import tqdm # 进度条工具
warnings.filterwarnings("ignore")
%matplotlib inline 

PATH = 'H:/learning_notes/project/kaggle/houseprice/data/'
# PATH = 'C:/Users/Dl/Documents/GitHub/learning_notes/project/kaggle/houseprice/data/'
train_data = pd.read_csv(PATH + 'train.csv')
train_data = train_data.replace({
    'Ex':5,
    'Gd':4,
    'TA':3,
    'Fa':2,
    'Po':1
})
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

Kaggle 分享的note基本都是对这些特征进行流水线式的处理，我之前也按他们的思路尝试过，思路大致如下：

填充缺失值 >>> 处理数值型特征（StandarScaler & np.log） >>> 处理类目型（pd.get_dummies)特征 >>> 建模（Lasso,KernelRidge,xgboost） >>> 验证（按月份划分测试与训练集用于本地验证） >>> 调参（GridSearchCV & RandomizedSearchCV） >>> 多模型融合（Voting & Blend）

最终得分停留在12.105，感觉这种处理方式很不好，做完了之后都想不起来自己干了个啥，所以我准备在这里将所有特征分文别类地建立一个属于自己的模型。

> 注：为了便于阐述建模思路，本Notebook不包含各类数据对价格的影响图表

### 区位因素

区位因素有：MSZoning;MSSubClass;Neighborhood

In [2]:
region_features = train_data[['MSSubClass','MSZoning','Neighborhood']]
y = np.log(train_data['SalePrice']/train_data['GrLivArea']) # 单价
region_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 3 columns):
MSSubClass      1460 non-null int64
MSZoning        1460 non-null object
Neighborhood    1460 non-null object
dtypes: int64(1), object(2)
memory usage: 34.3+ KB


In [3]:
region_features = region_features.astype(str) # 这三项都作为类目型特征进行处理
x = pd.get_dummies(region_features)
clf = RidgeCV(alphas=[1e-5,1e-4,1e-3,1e-2,1e-1])
clf.fit(x,y)
yp = clf.predict(x)
print(np.sqrt(mean_squared_error(yp,y)))
# x.info()

0.18121917586


效果惊人，这个八成过拟合了，这样验证效果是不合理的，应该将数据分为训练集和测试集，使用训练集训练模型，然后使用测试集检验效果。

最常见又简单的做法是使用sklearn.cross_validation.train_test_split()函数进行分割：

In [4]:
from sklearn.cross_validation import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.15,random_state=10) # 测试集大小随您喜欢，不过为了保证模型效果，训练集需要有足够的数据量
clf = RidgeCV(alphas=[1e-5,1e-4,1e-3,1e-2,1e-1]) # 重新定义模型，要不然就会直接使用上面已经建立好的模型了
clf.fit(x_train,y_train)
yp = clf.predict(x_test)
print(np.sqrt(mean_squared_error(yp,y_test))) 

0.19154207526


上述代码中，数据划分是随机的，因为这种随机性导致随着划分方式的不同会有不同的测试结果，需要多测几次求平均值，正好sklearn中还提供了一个更加方便的交叉验证函数:cross_val_score()，它会使用KFold方法将数据集划分我N个部分，然后用其中N-1个子集合并作为训练集，剩下的一个子集作为测试集，我们可以直接用这个函数来验证每一步的效果：

In [5]:
def check_data(x,y,cv=10):
    # clf =  RidgeCV(alphas=[1e-5,1e-4,1e-3,1e-2,1e-1])
    clf = KernelRidge(kernel='linear')
    scores = cross_val_score(clf,x,y,scoring='neg_mean_squared_error',cv = cv) # 返回的是负数
    scores = [np.sqrt((-1)*score) for score in scores]
    return np.mean(scores)
'得分:%.4f' % (100 * check_data(x,y)),'特征数量:%d' % len(x.columns)

('得分:19.2605', '特征数量:45')

### 年代因素

年代因素有：YearBuilt;YearRemodAdd;YrSold;

In [6]:
year_features = train_data[['YearBuilt','YearRemodAdd','YrSold','MoSold']] # 不同月份之间房价差异要大过不同年份，所以月份也是很重要的因素
year_features.info() # 查漏补缺

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 4 columns):
YearBuilt       1460 non-null int64
YearRemodAdd    1460 non-null int64
YrSold          1460 non-null int64
MoSold          1460 non-null int64
dtypes: int64(4)
memory usage: 45.7 KB


**捣鼓新特征**

In [7]:
year_features['Remodeled'] = (train_data['YearBuilt'] != train_data['YearRemodAdd']) * 1
year_features['Remodeled'].value_counts()

0    764
1    696
Name: Remodeled, dtype: int64

In [8]:
year_features['Age'] = train_data['YrSold'] - train_data['YearBuilt'] + 1
year_features['Age'].value_counts()

2      100
1       64
5       41
3       39
4       36
32      31
6       30
7       30
9       25
38      25
40      24
8       24
52      24
10      23
48      23
33      23
11      22
43      22
12      22
50      21
54      20
16      20
37      20
31      19
44      19
49      18
42      17
46      17
55      17
39      16
      ... 
72       3
73       3
75       3
24       3
107      3
96       3
101      3
76       2
65       2
128      2
118      2
111      2
64       2
105      1
130      1
129      1
104      1
127      1
126      1
123      1
120      1
103      1
116      1
115      1
112      1
80       1
110      1
136      1
108      1
137      1
Name: Age, Length: 122, dtype: int64

竟然有一百多年的房子在卖！！！

In [9]:
year_features['YrSold'] = year_features['YrSold'].astype(str)
year_features['MoSold'] = year_features['MoSold'].astype(str)
year_features = pd.get_dummies(year_features)

In [10]:
x = pd.concat([x,year_features],axis=1)
'得分:%.4f' % (100*check_data(x,y)),'特征数量:%d' % len(x.columns)

('得分:18.0132', '特征数量:66')

### 销售方式

不同的销售方式也会影响到价格，比如期房和现房价格不同，卖给亲戚和卖给陌生人价格也会有不同。

In [11]:
sale_features = train_data[['SaleType','SaleCondition']].astype(str) #这里其实本来就是str类型，但是还是加一下astype，免得出幺蛾子
sale_features = pd.get_dummies(sale_features)

In [12]:
x = pd.concat([x,sale_features],axis=1)
'得分:%.4f' % (100*check_data(x,y)),'特征数量:%d' % len(x.columns)

('得分:18.0915', '特征数量:81')

模型效果并没有得到改善

### 交通地形因素

In [13]:
traffic_landform = train_data[['LotFrontage','Street','Alley','LandContour','LandSlope','Condition1','Condition2']]
for f in ['LotFrontage','Street','Alley','LandContour','LandSlope','Condition1','Condition2']:
    traffic_landform[f] = traffic_landform[f].fillna(traffic_landform[f].mode()[0])
print(traffic_landform.info())
traffic_landform['LotFrontage'] = traffic_landform['LotFrontage'].fillna(traffic_landform['LotFrontage'].mean())
traffic_landform = pd.get_dummies(traffic_landform)
x = pd.concat([x,traffic_landform],axis=1)
'得分:%.4f' % (100*check_data(x,y)),'特征数量:%d' % len(x.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 7 columns):
LotFrontage    1460 non-null float64
Street         1460 non-null object
Alley          1460 non-null object
LandContour    1460 non-null object
LandSlope      1460 non-null object
Condition1     1460 non-null object
Condition2     1460 non-null object
dtypes: float64(1), object(6)
memory usage: 79.9+ KB
None


('得分:18.1439', '特征数量:110')

### 房屋总体特征

从外面一眼能看见的特征，买房子第一印象很重要

In [14]:
property_feature = train_data[['LotArea','LotShape','Utilities','LotConfig','BldgType','HouseStyle','OverallQual',
                               'OverallCond','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType',
                              'MasVnrArea','ExterQual','ExterCond','1stFlrSF','2ndFlrSF','GrLivArea','Functional']]
property_feature['MasVnrType'] = property_feature['MasVnrType'].fillna(property_feature['MasVnrType'].mode()[0])
property_feature['MasVnrArea'] = property_feature['MasVnrArea'].fillna(property_feature['MasVnrArea'].median())
property_feature = pd.get_dummies(property_feature)
x = pd.concat([x,property_feature],axis=1)
'得分:%.4f' % (100*check_data(x,y)),'特征数量:%d' % len(x.columns)

('得分:14.4589', '特征数量:199')

### 房屋内部配置

data_description.txt 中所描述的'Bedroom'和'Kitchen'两个字段并未出现在训练集中，我们中国人最关心的卧室数量，在这个数据集里面竟然丢失了！？？看看美国人买房子的时候都关心啥：地下室？？车库？？游泳池？？门廊？？

In [15]:
inner_feature = train_data[['Foundation','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinSF1','BsmtFinType2',
                           'BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','Heating','HeatingQC','CentralAir','Electrical',
                           'LowQualFinSF','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath',
                           'KitchenQual','TotRmsAbvGrd','Fireplaces','FireplaceQu','GarageType','GarageYrBlt',
                           'GarageFinish','GarageCars','GarageArea','GarageQual','GarageCond','PavedDrive','WoodDeckSF',
                           'OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','PoolQC','Fence','MiscFeature',
                           'MiscVal']]
# 很多数据有缺失，为了避免曲解数据含义，要一个一个填充
inner_feature[['BsmtQual','BsmtCond']] = inner_feature[['BsmtQual','BsmtCond']].fillna(0) # NAN 代表没有地下室，那么地下室质量可以置为最低级：0
inner_feature[['BsmtExposure','BsmtFinType1','BsmtFinType2']] = inner_feature[['BsmtExposure','BsmtFinType1','BsmtFinType2']].fillna('None')
inner_feature['Electrical'] = inner_feature['Electrical'].fillna(inner_feature['Electrical'].mode()[0])
inner_feature['FireplaceQu'] = inner_feature['FireplaceQu'].fillna(0.0)
inner_feature['GarageType'] = inner_feature['GarageType'].fillna('None')
inner_feature['GarageYrBlt'] = inner_feature['GarageYrBlt'].astype(str) # 年份，转化为str
inner_feature['GarageYrBlt'] = inner_feature['GarageYrBlt'].fillna('None')
inner_feature['GarageFinish'] = inner_feature['GarageFinish'].fillna('None')
inner_feature['GarageQual'] = inner_feature['GarageQual'].fillna(0)
inner_feature['GarageCond'] = inner_feature['GarageCond'].fillna(0)
inner_feature['PoolQC'] = inner_feature['PoolQC'].fillna(0)
inner_feature['Fence'] = inner_feature['Fence'].fillna('None')
inner_feature['MiscFeature'] = inner_feature['MiscFeature'].fillna('None')
inner_feature.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 41 columns):
Foundation       1460 non-null object
BsmtQual         1460 non-null float64
BsmtCond         1460 non-null float64
BsmtExposure     1460 non-null object
BsmtFinType1     1460 non-null object
BsmtFinSF1       1460 non-null int64
BsmtFinType2     1460 non-null object
BsmtFinSF2       1460 non-null int64
BsmtUnfSF        1460 non-null int64
TotalBsmtSF      1460 non-null int64
Heating          1460 non-null object
HeatingQC        1460 non-null int64
CentralAir       1460 non-null object
Electrical       1460 non-null object
LowQualFinSF     1460 non-null int64
BsmtFullBath     1460 non-null int64
BsmtHalfBath     1460 non-null int64
FullBath         1460 non-null int64
HalfBath         1460 non-null int64
KitchenQual      1460 non-null int64
TotRmsAbvGrd     1460 non-null int64
Fireplaces       1460 non-null int64
FireplaceQu      1460 non-null float64
GarageType       1460 non-nul

In [16]:
inner_feature = pd.get_dummies(inner_feature)
x = pd.concat([x,inner_feature],axis=1)
'得分:%.4f' % (100*check_data(x,y)),'特征数量:%d' % len(x.columns)

('得分:13.6306', '特征数量:387')

### 逐步向后CV

逐步地一个一个添加变量，如果新添加了特征之后模型得分得到改善则保留这个特征，否则去掉这个特征

In [17]:
x_new = pd.DataFrame()
first_feature = x.GrLivArea.values # 选一个很重要，肯定会包含在最终模型中的特征，我这里选择的是GrLivArea，也就是生活面积
x_new['GrLivArea'] = first_feature
x = x.drop('GrLivArea',axis=1)
cols = x.columns
init_score = check_data(x_new,y,cv=10) # 如果你用的电脑比较慢的话，可以把CV数量调低一点
for col in tqdm(cols):
    x_new[col] = x[col]
    new_score = check_data(x_new,y,cv=5)
    if new_score <= init_score:
        init_score = new_score # 添加特征并更新得分
        continue
    else:
        x_new = x_new.drop(col,axis=1) # 去除特征
x_new.info(),init_score

100%|█████████████████████████████| 386/386 [03:32<00:00,  1.82it/s]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Columns: 130 entries, GrLivArea to MiscFeature_Othr
dtypes: float64(2), int64(19), uint8(109)
memory usage: 395.0 KB


(None, 0.122636915562137)

最终选择了130个特征，得分12.2637,离理想的分数还有较大的差距。

### RandomizedSearchCV & GridSearchCV

特征工程之后我们要选择几个模型进行调参，为后面的模型融合做准备，这里选择RandomizedSearchCV或者GridSearchCV都可以，我之前用的都是GridSearchCV，所以这里要尝试一下RandomizedSearchCV的效果，随机也许能出奇迹。

初步选择的模型有KernelRidge,RandomForest,xgboost,SVR,GradientBoostingRegressor,Lasso六个模型

#### KernelRidge

In [39]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.grid_search import RandomizedSearchCV
from scipy.stats import uniform as sp_rand
krr = KernelRidge()
params = {
    'alpha':sp_rand(),
    'gamma':sp_rand(),
    'kernel':['linear','rbf','laplacian','polynomial','sigmoid'],
    'degree':range(1,4),
}
model1 = RandomizedSearchCV(krr,params,cv=10,scoring='neg_mean_squared_error')
model1.fit(x_new,y)
'best parameters:',model1.best_params_,np.sqrt((-1)*model1.best_score_)

('best parameters:',
 {'alpha': 0.42270389949504661,
  'degree': 1,
  'gamma': 0.62325187337886645,
  'kernel': 'polynomial'},
 0.12330216555560318)

('best parameters:',
 {'alpha': 0.8097910445350851,
  'degree': 1,
  'gamma': 0.67999184729440021,
  'kernel': 'polynomial'},
 0.12363014896491431)

#### RandomForestRegressor

In [40]:
# 这个模型训练会比较慢
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
params = {
    'n_estimators':[300,500,800,1000],
    'criterion':['mse'],
    'max_features':[0.5,'sqrt',0.3,0.8,'log2'],
    'min_samples_leaf':[2,3,4],
    'n_jobs':[-1],
    'max_depth':[3,5,7,9]
}
model2 = RandomizedSearchCV(rf,params,cv=10,scoring='neg_mean_squared_error')
model2.fit(x_new,y)
'best parameters:',model2.best_params_,np.sqrt((-1)*model2.best_score_)

('best parameters:',
 {'criterion': 'mse',
  'max_depth': 9,
  'max_features': 0.5,
  'min_samples_leaf': 2,
  'n_estimators': 800,
  'n_jobs': -1},
 0.14397710758247814)

('best parameters:',
 {'criterion': 'mse',
  'max_depth': 9,
  'max_features': 0.8,
  'min_samples_leaf': 2,
  'n_estimators': 300,
  'n_jobs': -1},
 0.14337035947616186)

#### xgboost

In [41]:
from xgboost import XGBRegressor
xgb = XGBRegressor()
params = {
    'learning_rate':[0.1],
    'max_depth':[2],
    'n_estimators':[500],
    'reg_alpha':[0.2,0.3,0.4,0.5,0.6],
    'reg_lambda':[0.2,0.3,0.4,0.5,0.6,0.7]
}
model3 = RandomizedSearchCV(xgb,params,cv=10,scoring='neg_mean_squared_error')
model3.fit(x_new,y)
'best parameters:',model3.best_params_,np.sqrt((-1)*model3.best_score_)

('best parameters:',
 {'learning_rate': 0.1,
  'max_depth': 2,
  'n_estimators': 500,
  'reg_alpha': 0.4,
  'reg_lambda': 0.3},
 0.12090706644835847)

('best parameters:',
 {'learning_rate': 0.1,
  'max_depth': 2,
  'n_estimators': 500,
  'reg_alpha': 0.5,
  'reg_lambda': 0.5},
 0.12106369526470077)

#### SVR

In [ ]:
'''from sklearn.svm import SVR
from sklearn.grid_search import GridSearchCV
sv = SVR(kernel='linear')
model = GridSearchCV(sv, cv=10,
                   param_grid={"C": [1e0, 1e-1, 1e-2, 1e-3],
                               "degree":[2,3,4]},scoring='neg_mean_squared_error')
model.fit(x_new,y)
'best parameters:',model.best_params_,np.sqrt((-1)*model.best_score_)'''

In [21]:
'''from sklearn.svm import SVR
from sklearn.grid_search import GridSearchCV
sv = SVR()
# Kernel 不要使用'precomputed'参数，因为precomputed这个核函数要求X为方阵
params = {
    'C':[1e-2,1e-1,1],
    'epsilon':[0.01,0.1],
    'kernel':['rbf','poly','sigmoid'],
    'gamma':[1e-2,1e-1]
}
model = GridSearchCV(sv,params,scoring='neg_mean_squared_error')
# model = RandomizedSearchCV(sv,params,cv=10,scoring='neg_mean_squared_error')
model.fit(x_new,y)
'best parameters:',model.best_params_,np.sqrt((-1)*model.best_score_)'''

"from sklearn.svm import SVR\nfrom sklearn.grid_search import GridSearchCV\nsv = SVR()\n# Kernel 不要使用'precomputed'参数，因为precomputed这个核函数要求X为方阵\nparams = {\n    'C':[1e-2,1e-1,1],\n    'epsilon':[0.01,0.1],\n    'kernel':['rbf','poly','sigmoid'],\n    'gamma':[1e-2,1e-1]\n}\nmodel = GridSearchCV(sv,params,scoring='neg_mean_squared_error')\n# model = RandomizedSearchCV(sv,params,cv=10,scoring='neg_mean_squared_error')\nmodel.fit(x_new,y)\n'best parameters:',model.best_params_,np.sqrt((-1)*model.best_score_)"

#### GradientBoostingRegressor

In [42]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.grid_search import GridSearchCV
gb = GradientBoostingRegressor()
params = {
    'loss':['ls', 'lad', 'huber', 'quantile'],
    'learning_rate':[1e-4,1e-3,1e-2,1e-1],
    'n_estimators':[100,200,400],
    'criterion':['mse'],
    'max_features':['sqrt','log2']
}
model4 = GridSearchCV(gb,params,scoring='neg_mean_squared_error')
model4.fit(x_new,y)
'best parameters:',model4.best_params_,np.sqrt((-1)*model4.best_score_)

('best parameters:',
 {'criterion': 'mse',
  'learning_rate': 0.1,
  'loss': 'huber',
  'max_features': 'sqrt',
  'n_estimators': 400},
 0.12893263683942019)

('best parameters:',
 {'criterion': 'mse',
  'learning_rate': 0.1,
  'loss': 'huber',
  'max_features': 'sqrt',
  'n_estimators': 400},
 0.12741224009492616)

In [37]:
from sklearn.linear_model import Ridge,Lasso
las = Lasso()
params = {
    'alpha':[1e-5,1e-4,1e-3,1e-2,1e-1,1],
    'fit_intercept':[True,False],
    'normalize':[True,False],
    'max_iter':[1e1,1e2,1e3]
}
model5 = GridSearchCV(rid,params,scoring='neg_mean_squared_error')
model5.fit(x_new,y)
'best parameters:',model5.best_params_,np.sqrt((-1)*model5.best_score_)

('best parameters:',
 {'alpha': 1e-05,
  'fit_intercept': True,
  'max_iter': 1000.0,
  'normalize': False},
 0.1233796262385489)

('best parameters:',
 {'alpha': 1e-05,
  'fit_intercept': True,
  'max_iter': 1000.0,
  'normalize': False},
 0.1233796262385489)
 
 其实这里使用LassoCV也行

得到模型：
{
    krr:12.363,
    rf:14.337,
    xgb:12.106,
    gb:12.741,
    las:12.338
}
姑且把这五个模型融合一下看看效果，虽然randomforest效果比较差，考虑到这个模型的口碑，还是尝试带入一下。

## 表决融合

In [38]:
model_weights = {
    model1:0.2,
    model2:0.1,
    model3:0.25,
    model4:0.2,
    model5:0.25
}
def voting_model(model_weights,x_new,y):
    yp = np.array([0 for i in range(len(y))]).reshape(-1,1)
    for k,v in model_weights.items():
        yp_new = k.predict(x_new)
        yp += yp_new * v
    score = np.sqrt(mean_squared_error(yp,y))
    return score
voting_model(model_weights,x_new,y)

NotFittedError: This KernelRidge instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.